## 
1. Data Augmentation
2. Logistic Regression(non-exercise/exercise)
3. exercise -> Ensemble(LSTM + CNN) + Stratified K Fold

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 기본 directory 설정
import os
os.chdir('/content/drive/MyDrive/Monthly_Workout')

In [1]:
# 모듈 불러오기
import random
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# 데이터 불러오기
path = './' # 기본 directory 경로에 추가 할 경로

train = pd.read_csv(path + 'train_features.csv')
train_labels = pd.read_csv(path + 'train_labels.csv')
test = pd.read_csv(path + 'test_features.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

In [3]:
act_list=train.iloc[:,2:].columns
acc_list=['acc_x','acc_y','acc_z']
gy_list=['gy_x','gy_y','gy_z']
act_list

Index(['acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z'], dtype='object')

In [4]:
from sklearn.preprocessing import StandardScaler
# Scaling
scaler=StandardScaler()
train[act_list]=scaler.fit_transform(train[act_list])
test[act_list]=scaler.transform(test[act_list])
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007
1,0,1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459
2,0,2,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767
3,0,3,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232
4,0,4,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.367432,-0.845648,0.503707,-0.461266,-1.413128,-1.092639
1874996,3124,596,-0.326315,-0.845833,0.578875,-0.474109,-1.382884,-1.097377
1874997,3124,597,-0.300794,-0.859728,0.652722,-0.436694,-1.341672,-1.136169
1874998,3124,598,-0.253120,-0.886873,0.673002,-0.367051,-1.346630,-1.148831


In [5]:
# acc 데이터와 gy 데이터로 분할
def sensor_split(data):
    X_acc = []
    X_gy = []

    for i in tqdm(data['id'].unique()):
        temp_acc = np.array(data[data['id'] == i].loc[:,acc_list])
        temp_gy = np.array(data[data['id'] == i].loc[:,gy_list])
        X_acc.append(temp_acc)
        X_gy.append(temp_gy)
      
    X_acc = np.array(X_acc).reshape(-1,600,3)
    X_gy = np.array(X_gy).reshape(-1,600,3)

    return X_acc, X_gy

In [6]:
# acc, gy 데이터 분할
X_train_mod=pd.merge(train,train_labels,how='left',on='id')
X_train_acc, X_train_gy= sensor_split(X_train_mod)

X_train_acc.shape, X_train_gy.shape

100%|█████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:20<00:00, 153.77it/s]


((3125, 600, 3), (3125, 600, 3))

In [7]:
# 데이터 증강

def Permutation(X, nPerm=4, minSegLength=10):
    X_new = np.zeros(X.shape)
    idx = np.random.permutation(nPerm)
    bWhile = True
    while bWhile == True:
        segs = np.zeros(nPerm+1, dtype=int)
        segs[1:-1] = np.sort(np.random.randint(minSegLength, X.shape[0]-minSegLength, nPerm-1))
        segs[-1] = X.shape[0]
        if np.min(segs[1:]-segs[0:-1]) > minSegLength:
            bWhile = False
    pp = 0
    for ii in range(nPerm):
        x_temp = X[segs[idx[ii]]:segs[idx[ii]+1],:]
        X_new[pp:pp+len(x_temp),:] = x_temp
        pp += len(x_temp)
    return (X_new)

def aug(data, uid, shift):
    shift_data = np.roll(data[uid], shift, axis=0)
    return shift_data
def Rolling(data):
    aug_data=[]
    for i in range(data.shape[0]):
        temp=list((aug(data,i,int(random.random()*600))))
        aug_data.append(temp)
    return np.array(aug_data)

In [8]:
# 데이터 증강 (반복하고 싶은 만큼 조정)
X_train_acc_temp = X_train_acc.copy()
X_train_gy_temp = X_train_gy.copy()

y_train = train_labels['label']
y_train_total = np.append(y_train, y_train, axis=0)

rep = 3
for i in range(rep):
    X_train_acc_roll = Rolling(X_train_acc_temp)
    X_train_acc_rp = Permutation(Rolling(X_train_acc_temp)) # rolling + permutation

    X_train_gy_roll = Rolling(X_train_gy_temp)
    X_train_gy_rp = Permutation(Rolling(X_train_gy_temp)) # rolling + permutation

    # 증강시킨 데이터 원래 데이터에 추가
    X_train_acc = np.append(X_train_acc, X_train_acc_roll, axis=0)
    X_train_acc = np.append(X_train_acc, X_train_acc_rp, axis=0)

    X_train_gy = np.append(X_train_gy, X_train_gy_roll, axis=0)
    X_train_gy = np.append(X_train_gy, X_train_gy_rp, axis=0)

    y_train_total = np.append(y_train_total, y_train, axis=0)
    if i != (rep-1): # 마지막 한 번 제외
        y_train_total = np.append(y_train_total, y_train, axis=0)
        
X_train_acc.shape, X_train_gy.shape, y_train_total.shape

((21875, 600, 3), (21875, 600, 3), (21875,))

In [9]:
# np array 형태를 dataframe 으로 변환
acc = [e for sl in X_train_acc for e in sl]

df_report = np.stack(acc, axis = 0)
df_acc = pd.DataFrame(df_report, columns= ['acc_x', 'acc_y', 'acc_z']) 

df_acc

,acc_x,acc_y,acc_z
0,2.307314,-0.029939,-0.432104
1,2.421086,-0.063321,-0.504058
2,2.444664,-0.056749,-0.654199
3,2.428612,-0.116782,-0.573407
4,2.439475,-0.044220,-0.588886
...,...,...,...
13124995,-0.554249,-0.556933,0.962225
13124996,-0.949177,-0.482134,0.836416
13124997,-1.075873,-0.304617,0.803861
13124998,-0.906468,-0.163168,0.949040


In [10]:
# gy
gy = [e for sl in X_train_gy for e in sl]

df_report = np.stack(gy, axis = 0)
df_gy = pd.DataFrame(df_report, columns= ['gy_x', 'gy_y', 'gy_z']) 

df_gy

,gy_x,gy_y,gy_z
0,0.011776,-0.410859,-0.461007
1,0.026484,-0.528052,-0.365459
2,-0.037962,-0.596041,-0.366767
3,0.066099,-0.725311,-0.401232
4,0.091969,-0.784094,-0.408413
...,...,...,...
13124995,0.117360,-0.358365,-0.620826
13124996,-0.009031,-0.347483,-0.521413
13124997,0.221687,-0.400332,-0.461006
13124998,0.071811,-0.370874,-0.684420


In [11]:
# acc, gy 데이터프레임 병합
df_aug_result = pd.concat([df_acc, df_gy], axis = 1)

train = df_aug_result
train

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007
1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459
2,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767
3,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232
4,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413
...,...,...,...,...,...,...
13124995,-0.554249,-0.556933,0.962225,0.117360,-0.358365,-0.620826
13124996,-0.949177,-0.482134,0.836416,-0.009031,-0.347483,-0.521413
13124997,-1.075873,-0.304617,0.803861,0.221687,-0.400332,-0.461006
13124998,-0.906468,-0.163168,0.949040,0.071811,-0.370874,-0.684420


In [12]:
train['acc_t'] = (train['acc_x'] ** 2) + (train['acc_y'] ** 2) + (train['acc_z'] ** 2) ** (1/3)
test['acc_t'] = (test['acc_x'] ** 2) + (test['acc_y'] ** 2) + (test['acc_z'] ** 2) ** (1/3)

train['gy_t'] = (train['gy_x'] ** 2) + (train['gy_y'] ** 2) + (train['gy_z'] ** 2) ** (1/3)
test['gy_t'] = (test['gy_x'] ** 2) + (test['gy_y'] ** 2) + (test['gy_z'] ** 2) ** (1/3)

# Signal 극대화 (peak 캐치 유용)
train['acc_mag'] = (train['acc_x'] ** 2) + (train['acc_y'] ** 2) + (train['acc_z'] ** 2)
test['acc_mag'] = (test['acc_x'] ** 2) + (test['acc_y'] ** 2) + (test['acc_z'] ** 2)

train['gy_mag'] = (train['gy_x'] ** 2) + (train['gy_y'] ** 2) + (train['gy_z'] ** 2)
test['gy_mag'] = (test['gy_x'] ** 2) + (test['gy_y'] ** 2) + (test['gy_z'] ** 2)

In [13]:
# vector
train['acc_vector'] = np.sqrt((train['acc_x'] ** 2) +(train['acc_y'] ** 2)+(train['acc_z'] ** 2))
test['acc_vector'] = np.sqrt((test['acc_x'] ** 2) +(test['acc_y'] ** 2)+(test['acc_z'] ** 2))

train['gy_vector'] = np.sqrt((train['gy_x'] ** 2) +(train['gy_y'] ** 2)+(train['gy_z'] ** 2))
test['gy_vector'] = np.sqrt((test['gy_x'] ** 2) +(test['gy_y'] ** 2)+(test['gy_z'] ** 2))

train['acc_YZvector'] = np.sqrt((train['acc_y'] ** 2) +(train['acc_z'] ** 2))
test['acc_YZvector'] = np.sqrt((test['acc_y'] ** 2) +(test['acc_z'] ** 2))

train['gy_YZvector'] = np.sqrt((train['gy_y'] ** 2) +(train['gy_z'] ** 2))
test['gy_YZvector'] = np.sqrt((test['gy_y'] ** 2) +(test['gy_z'] ** 2))

train['acc_XYvector'] = np.sqrt((train['acc_x'] ** 2) +(train['acc_y'] ** 2))
test['acc_XYvector'] = np.sqrt((test['acc_x'] ** 2) +(test['acc_y'] ** 2))

train['gy_XYvector'] = np.sqrt((train['gy_x'] ** 2) +(train['gy_y'] ** 2))
test['gy_XYvector'] = np.sqrt((test['gy_x'] ** 2) +(test['gy_y'] ** 2))

train['acc_XZvector'] = np.sqrt((train['acc_x'] ** 2) +(train['acc_z'] ** 2))
test['acc_XZvector'] = np.sqrt((test['acc_x'] ** 2) +(test['acc_z'] ** 2))

train['gy_XZvector'] = np.sqrt((train['gy_x'] ** 2) +(train['gy_z'] ** 2))
test['gy_XZvector'] = np.sqrt((test['gy_x'] ** 2) +(test['gy_z'] ** 2))

# 자이로스코프 무게중심
train['gy_Centerofgravity'] = (train['gy_x']+train['gy_y']+train['gy_z'])/3
test['gy_Centerofgravity'] = (test['gy_x']+test['gy_y']+test['gy_z'])/3

# roll & pitch
train['roll'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2))
test['roll'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2))

train['pitch'] = np.arctan(train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2))
test['pitch'] = np.arctan(test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2))

In [14]:
from math import pi

train['math_roll'] = np.arctan(- train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2)) * (180/pi)
test['math_roll'] = np.arctan(- test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2)) * (180/pi)

train['math_pitch'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2)) * (180/pi)
test['math_pitch'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2)) * (180/pi)

train['gy_roll'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2))
test['gy_roll'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2))

train['gy_pitch'] = np.arctan(train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2))
test['gy_pitch'] = np.arctan(test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2))

train['gy_math_roll'] = np.arctan(- train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2)) * (180/pi)
test['gy_math_roll'] = np.arctan(- test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2)) * (180/pi)

train['gy_math_pitch'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2)) * (180/pi)
test['gy_math_pitch'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2)) * (180/pi)

print(train.shape)
train

(13125000, 27)


,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_t,gy_t,acc_mag,gy_mag,...,gy_XZvector,gy_Centerofgravity,roll,pitch,math_roll,math_pitch,gy_roll,gy_pitch,gy_math_roll,gy_math_pitch
0,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007,5.896150,0.765711,5.511308,0.381471,...,0.461158,-0.286697,-0.012753,1.385231,-79.367910,-0.730704,-0.727781,0.019068,-1.092507,-41.698765
1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459,6.499033,0.790703,6.119743,0.413101,...,0.366417,-0.289009,-0.025599,1.363966,-78.149478,-1.466742,-0.964174,0.041217,-2.361573,-55.243076
2,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767,6.733202,0.869088,6.407580,0.491224,...,0.368727,-0.333590,-0.022421,1.308383,-74.964812,-1.284609,-1.016793,-0.054190,3.104851,-58.257969
3,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232,6.601997,1.074443,6.240592,0.691433,...,0.406641,-0.353481,-0.046765,1.334352,-76.452715,-2.679451,-1.059819,0.079576,-4.559348,-60.723136
4,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413,6.655560,1.173732,6.299780,0.790064,...,0.418640,-0.366846,-0.017619,1.333287,-76.391740,-1.009478,-1.080386,0.103655,-5.938975,-61.901532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13124995,-0.554249,-0.556933,0.962225,0.117360,-0.358365,-0.620826,1.592021,0.869944,1.543242,0.527623,...,0.631821,-0.287277,-0.464882,-0.462467,26.497389,-26.635798,-0.515948,0.162280,-9.297970,-29.561615
13124996,-0.949177,-0.482134,0.836416,-0.009031,-0.347483,-0.521413,2.021122,0.768646,1.832982,0.392697,...,0.521491,-0.292642,-0.364105,-0.776912,44.513792,-20.861694,-0.587766,-0.014412,0.825765,-33.676511
13124997,-1.075873,-0.304617,0.803861,0.221687,-0.400332,-0.461006,2.114839,0.806178,1.896487,0.421938,...,0.511539,-0.213217,-0.223042,-0.896655,51.374528,-12.779342,-0.664043,0.348283,-19.955151,-38.046869
13124998,-0.906468,-0.163168,0.949040,0.071811,-0.370874,-0.684420,1.814039,0.919336,1.748985,0.611135,...,0.688177,-0.327828,-0.123695,-0.755186,43.268982,-7.087176,-0.494298,0.091989,-5.270578,-28.321201


In [15]:
test

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_t,gy_t,...,gy_XZvector,gy_Centerofgravity,roll,pitch,math_roll,math_pitch,gy_roll,gy_pitch,gy_math_roll,gy_math_pitch
0,3125,0,-0.249727,0.002786,0.202699,0.837933,1.212439,0.179961,0.407437,2.490891,...,0.857040,0.743444,0.008662,-0.888926,50.931703,0.496277,0.955473,0.599650,-34.357431,54.744571
1,3125,1,-0.018931,0.296752,-0.231628,0.957733,1.318163,0.217914,0.465582,3.016930,...,0.982211,0.831270,0.906416,-0.050246,2.878862,51.933790,0.930415,0.621937,-35.634339,53.308876
2,3125,2,0.119178,0.119214,-0.393070,0.507385,1.286118,0.176158,0.565010,2.225783,...,0.537095,0.656553,0.282480,0.282393,-16.179926,16.184928,1.175202,0.372602,-21.348525,67.334116
3,3125,3,0.135533,-0.132967,-0.526513,0.410870,1.214796,0.177867,0.688087,1.960818,...,0.447717,0.601178,-0.239861,0.244583,-14.013578,-13.743013,1.217689,0.322938,-18.502983,69.768460
4,3125,4,0.189646,0.064763,-0.792275,0.357015,1.170034,0.184028,0.896377,1.819974,...,0.401654,0.570359,0.079331,0.234196,-13.418416,4.545333,1.240117,0.292765,-16.774220,71.053458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469195,3906,595,0.771160,-1.061284,1.235599,-0.150699,0.198219,-2.707576,2.872478,2.004618,...,2.711767,-0.886685,-0.629699,0.442182,-25.335189,-36.079082,0.072966,-0.055453,3.177212,4.180657
469196,3906,596,0.769913,-1.016963,1.185170,-0.395387,-0.110361,-2.566486,2.746900,2.043041,...,2.596763,-1.024078,-0.623740,0.458035,-26.243449,-35.737662,-0.042474,-0.152717,8.750045,-2.433565
469197,3906,597,0.804762,-1.000674,1.122728,-0.437428,-0.310409,-2.314917,2.729220,2.037639,...,2.355883,-1.020918,-0.626922,0.491331,-28.151182,-35.919995,-0.131005,-0.185140,10.607717,-7.506009
469198,3906,598,0.771076,-0.904187,1.095665,-0.436282,-0.408929,-1.977082,2.474912,1.932815,...,2.024647,-0.940764,-0.593661,0.497292,-28.492706,-34.014293,-0.199294,-0.212823,12.193862,-11.418717


# simple Logistic

In [16]:
!pip install torch

In [17]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

print("Device", device)

Device cpu


In [18]:
train.head(3)

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_t,gy_t,acc_mag,gy_mag,...,gy_XZvector,gy_Centerofgravity,roll,pitch,math_roll,math_pitch,gy_roll,gy_pitch,gy_math_roll,gy_math_pitch
0,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007,5.896150,0.765711,5.511308,0.381471,...,0.461158,-0.286697,-0.012753,1.385231,-79.367910,-0.730704,-0.727781,0.019068,-1.092507,-41.698765
1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459,6.499033,0.790703,6.119743,0.413101,...,0.366417,-0.289009,-0.025599,1.363966,-78.149478,-1.466742,-0.964174,0.041217,-2.361573,-55.243076
2,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767,6.733202,0.869088,6.407580,0.491224,...,0.368727,-0.333590,-0.022421,1.308383,-74.964812,-1.284609,-1.016793,-0.054190,3.104851,-58.257969


In [19]:
t = train.values
t = np.array(t)
t = t.reshape(21875, 600*27)
t

array([[ 2.30731395e+00, -2.99388140e-02, -4.32103789e-01, ...,
        -4.21042460e-01,  2.41239560e+01,  3.09488225e+01],
       [ 3.30643980e-01,  1.41402394e-01,  1.69088220e+00, ...,
         9.61075676e-01, -5.50655800e+01, -3.28493932e+01],
       [ 9.62426621e-01, -4.70169843e-01,  1.56933474e+00, ...,
         3.61830361e-01, -2.07313526e+01,  6.87375298e+01],
       ...,
       [ 1.62335106e+00, -1.20820099e+00, -2.25520528e-02, ...,
         8.43045699e-01, -4.83029605e+01, -1.94274261e+01],
       [-1.29857737e+00,  1.63588732e+00, -1.35234876e+00, ...,
        -5.51475835e-01,  3.15972378e+01,  1.45186769e+00],
       [-4.62685849e-01, -3.66571738e-01,  7.59722068e-01, ...,
        -4.93552271e-01,  2.82784621e+01, -1.59021610e+01]])

In [20]:
out_list = np.expand_dims(np.array(t), axis = 1)
data = torch.from_numpy(out_list)
data = data.squeeze(1)
data

tensor([[ 2.3073e+00, -2.9939e-02, -4.3210e-01,  ..., -4.2104e-01,
          2.4124e+01,  3.0949e+01],
        [ 3.3064e-01,  1.4140e-01,  1.6909e+00,  ...,  9.6108e-01,
         -5.5066e+01, -3.2849e+01],
        [ 9.6243e-01, -4.7017e-01,  1.5693e+00,  ...,  3.6183e-01,
         -2.0731e+01,  6.8738e+01],
        ...,
        [ 1.6234e+00, -1.2082e+00, -2.2552e-02,  ...,  8.4305e-01,
         -4.8303e+01, -1.9427e+01],
        [-1.2986e+00,  1.6359e+00, -1.3523e+00,  ..., -5.5148e-01,
          3.1597e+01,  1.4519e+00],
        [-4.6269e-01, -3.6657e-01,  7.5972e-01,  ..., -4.9355e-01,
          2.8278e+01, -1.5902e+01]], dtype=torch.float64)

In [21]:
# out_list = train_labels['label']
# out_list = np.array(out_list)
# label = torch.from_numpy(out_list)
# label

In [22]:
# train_label.loc[(train_label['label'] != 26)] = 1
# train_label.loc[(train_label['label'] == 26)] = 0
# train_label

In [23]:
y_train_total[y_train_total != 26] = 1
y_train_total[y_train_total == 26] = 0
y_train_total

array([1, 0, 1, ..., 1, 0, 1], dtype=int64)

In [24]:
out_list = y_train_total
out_list = np.array(out_list)
label = torch.from_numpy(out_list)
label

tensor([1, 0, 1,  ..., 1, 0, 1])

In [ ]:
# ## out_list = []
# for i in range(15625):
#     id = train_data.loc[(train_data['id'] == i)].values[:, 2:]
#     id = np.reshape(id, (-1, 600 * 27))
#     out_list.append(id)

# out_list = np.array(out_list)
# #out_list.astype(tf.float64)
# out_list = np.expand_dims(np.array(out_list), axis = 1)
# data = torch.from_numpy(out_list)
# data = data.squeeze(1)

In [ ]:
# out_list = []
# for i in range(15625):
#     label = train_labels.loc[(train_labels['id'] == i)].values[0,1] 
#     out_list.append(label)

# out_list = np.array(out_list)
# #out_list = np.expand_dims(np.array(out_list), axis = 1)
# label = torch.from_numpy(out_list)

# len(data), len(label)

In [25]:
len(y_train_total[y_train_total == 1]), len(y_train_total[y_train_total == 0])

(11249, 10626)

In [26]:
label[:100]

tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0])

In [27]:
len(data), len(label)

(21875, 21875)

In [131]:
class Classify(nn.Module):

    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(16200, 8100) # 3600 1800
        self.linear2 = nn.Linear(8100, 2000) # 1800 900
        self.linear3 = nn.Linear(2000, 500) # 900 300
        self.linear4 = nn.Linear(500, 0) # 300 100
        self.linear5 = nn.Linear(0, 1) # 100 20

        # self.Softmax = nn.Softmax()

    def forward(self, x):
        out = F.relu(self.linear(x))
        out = F.relu(self.linear2(out))
        out = F.relu(self.linear3(out))
        out = F.relu(self.linear4(out))
        out = F.relu(self.linear5(out))
        # out = self.Softmax(out)

        #print("out.shape", out.shape)
        return out

In [ ]:
class Classify(nn.Module):

    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(16200, 1) # 3600 1800
#         self.linear2 = nn.Linear(8100, 2000) # 1800 900
#         self.linear3 = nn.Linear(2000, 500) # 900 300
#         self.linear4 = nn.Linear(500, 0) # 300 100
#         self.linear5 = nn.Linear(0, 1) # 100 20
        self.Softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
#         out = F.relu(self.linear(x))
#         out = F.relu(self.linear2(out))
#         out = F.relu(self.linear3(out))
#         out = F.relu(self.linear4(out))
#         out = F.relu(self.linear5(out))
        out = self.Softmax(out)
        return self.sigmoid(out)

        #print("out.shape", out.shape)
        #return out

In [142]:
model = Classify()

optimizer = optim.SGD(model.parameters(), lr=0.01)
#loss = nn.CrossEntropyLoss()
batch_size = 600
num_epochs = 10

In [143]:
#20분정도 소요

from torch.utils.data import TensorDataset

train_dataset = TensorDataset(data, label)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

for epoch in range(num_epochs):
    avg_cost = 0
    batch_length = len(train_loader)
    for x, y in tqdm(train_loader):
        x = x.squeeze(1)
        #print(x)
        #print(x.shape, y.shape)
        pred = model(x.float())
        
        #print("pred 1", pred.shape)
        pred = pred.squeeze(1)
        #print("pred 2", pred.shape)

        #print(pred.shape)
        #print("y.shape :", y.shape)
        #      cost = loss(pred, y)

        cost = F.binary_cross_entropy(pred, y.float())

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

#         corr = torch.argmax(pred)
#         num_correct = (corr == y).sum().item()
#         avg_cost += cost / batch_length
#         acc = num_correct / batch_length
    #print(cost, corr, num_correct, batch_length)
        
    #print("Accuracy", acc * 100)
    #print("Average Cost", avg_cost)
    
#     if epoch % 10 == 0:
    prediction = pred >= torch.FloatTensor([0.5])

    correct_prediction = prediction.float() == y
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
        epoch+1, num_epochs, cost.item(), accuracy * 100,
    ))
prediction = pred >= torch.FloatTensor([0.5])
print(prediction)

  8%|██████▉                                                                            | 3/36 [00:00<00:01, 23.14it/s]

Epoch    1/10 Cost: 46.564156 Accuracy 52.33%


  8%|██████▉                                                                            | 3/36 [00:00<00:01, 21.96it/s]

Epoch    2/10 Cost: 41.249676 Accuracy 58.33%


  6%|████▌                                                                              | 2/36 [00:00<00:01, 18.92it/s]

...

Epoch    8/10 Cost: 37.441799 Accuracy 61.83%


  6%|████▌                                                                              | 2/36 [00:00<00:01, 19.86it/s]

Epoch    9/10 Cost: 38.396309 Accuracy 61.17%


100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:01<00:00, 20.51it/s]

Epoch   10/10 Cost: 33.153496 Accuracy 66.50%
tensor([ True,  True,  True, False, False, False,  True, False, False,  True,
        False, False,  True, False,  True, False, False,  True, False,  True,
        False, False,  True, False,  True,  True, False, False, False, False,
        False,  True, False,  True, False, False, False,  True,  True, False,
        False, False,  True, False, False,  True, False, False,  True, False,
         True, False,  True,  True,  True,  True, False, False, False, False,
        False, False,  True,  True, False, False, False,  True,  True, False,
        False,  True, False,  True,  True, False,  True, False,  True,  True,
        False, False, False,  True,  True,  True, False, False, False, False,
         True, False,  True,  True,  True,  True, False, False,  True,  True,
         True, False,  True,  True,  True,  True,  True, False, False,  True,
        False, False, False,  True,  True,  True,  True, False,  True, False,
         True, Fal

In [106]:
# #20분정도 소요

# x_train = data
# y_train = label

# num_epochs = 10

# W = torch.zeros((21875, 27), requires_grad=True)
# b = torch.zeros(27, requires_grad=True)

# optimizer = optim.SGD([W, b], lr=1)

# for epoch in range(num_epochs):
#     x_train = x_train.squeeze(1)

#     # Cost 계산
#     x_train = x_train.float()
#     hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
#     cost = F.binary_cross_entropy(hypothesis, y_train)
#     #cost = F.binary_cross_entropy(pred, y.float())

#     optimizer.zero_grad()
#     cost.backward()
#     optimizer.step()

#     print('Epoch {:4d}/{}'.format(epoch+1, num_epochs))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (21875x16200 and 21875x27)

In [38]:
torch.save(model.state_dict(), path + '211007model.pt')
torch.save(model.state_dict(), path + '211007model_state_dict.pt')

In [ ]:
modell = torch.load(path + '211007model.pt')
modell.load_state_dict(torch.load(path + '211007model_state_dict.pt'))

In [34]:
test.iloc[:, 2:]

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_t,gy_t,acc_mag,gy_mag,...,gy_XZvector,gy_Centerofgravity,roll,pitch,math_roll,math_pitch,gy_roll,gy_pitch,gy_math_roll,gy_math_pitch
0,-0.249727,0.002786,0.202699,0.837933,1.212439,0.179961,0.407437,2.490891,0.103458,2.204526,...,0.857040,0.743444,0.008662,-0.888926,50.931703,0.496277,0.955473,0.599650,-34.357431,54.744571
1,-0.018931,0.296752,-0.231628,0.957733,1.318163,0.217914,0.465582,3.016930,0.142072,2.702292,...,0.982211,0.831270,0.906416,-0.050246,2.878862,51.933790,0.930415,0.621937,-35.634339,53.308876
2,0.119178,0.119214,-0.393070,0.507385,1.286118,0.176158,0.565010,2.225783,0.182919,1.942569,...,0.537095,0.656553,0.282480,0.282393,-16.179926,16.184928,1.175202,0.372602,-21.348525,67.334116
3,0.135533,-0.132967,-0.526513,0.410870,1.214796,0.177867,0.688087,1.960818,0.313265,1.676181,...,0.447717,0.601178,-0.239861,0.244583,-14.013578,-13.743013,1.217689,0.322938,-18.502983,69.768460
4,0.189646,0.064763,-0.792275,0.357015,1.170034,0.184028,0.896377,1.819974,0.667859,1.530305,...,0.401654,0.570359,0.079331,0.234196,-13.418416,4.545333,1.240117,0.292765,-16.774220,71.053458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469195,0.771160,-1.061284,1.235599,-0.150699,0.198219,-2.707576,2.872478,2.004618,3.247715,7.392969,...,2.711767,-0.886685,-0.629699,0.442182,-25.335189,-36.079082,0.072966,-0.055453,3.177212,4.180657
469196,0.769913,-1.016963,1.185170,-0.395387,-0.110361,-2.566486,2.746900,2.043041,3.031608,6.755359,...,2.596763,-1.024078,-0.623740,0.458035,-26.243449,-35.737662,-0.042474,-0.152717,8.750045,-2.433565
469197,0.804762,-1.000674,1.122728,-0.437428,-0.310409,-2.314917,2.729220,2.037639,2.909507,5.646540,...,2.355883,-1.020918,-0.626922,0.491331,-28.151182,-35.919995,-0.131005,-0.185140,10.607717,-7.506009
469198,0.771076,-0.904187,1.095665,-0.436282,-0.408929,-1.977082,2.474912,1.932815,2.612593,4.266417,...,2.024647,-0.940764,-0.593661,0.497292,-28.492706,-34.014293,-0.199294,-0.212823,12.193862,-11.418717


In [35]:
t = test.iloc[:, 2:].values
t = np.array(t)
t = t.reshape(782, 600*27)

out_list = np.expand_dims(np.array(t), axis = 1)
tdata = torch.from_numpy(out_list)
tdata = tdata.squeeze(1)
tdata

tensor([[-2.4973e-01,  2.7860e-03,  2.0270e-01,  ..., -1.2808e+00,
          7.3385e+01,  1.2077e+01],
        [ 1.0500e+00,  2.8800e+00, -8.3385e-01,  ..., -7.7667e-01,
          4.4500e+01,  1.5755e+01],
        [ 1.5147e+00,  1.5138e+00, -4.3767e-01,  ...,  1.0855e+00,
         -6.2196e+01,  3.4494e+00],
        ...,
        [-8.2321e-01,  1.7031e-01, -1.8111e-01,  ..., -1.1133e+00,
          6.3787e+01, -3.6912e+00],
        [ 2.2491e+00, -3.5514e-01, -9.3840e-01,  ..., -7.6773e-02,
          4.3988e+00,  6.5135e+01],
        [-7.3612e-01, -3.3655e-02, -3.4829e-01,  ..., -2.6492e-01,
          1.5179e+01, -1.5947e+01]], dtype=torch.float64)

In [36]:
test.iloc[:, 2:]

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_t,gy_t,acc_mag,gy_mag,...,gy_XZvector,gy_Centerofgravity,roll,pitch,math_roll,math_pitch,gy_roll,gy_pitch,gy_math_roll,gy_math_pitch
0,-0.249727,0.002786,0.202699,0.837933,1.212439,0.179961,0.407437,2.490891,0.103458,2.204526,...,0.857040,0.743444,0.008662,-0.888926,50.931703,0.496277,0.955473,0.599650,-34.357431,54.744571
1,-0.018931,0.296752,-0.231628,0.957733,1.318163,0.217914,0.465582,3.016930,0.142072,2.702292,...,0.982211,0.831270,0.906416,-0.050246,2.878862,51.933790,0.930415,0.621937,-35.634339,53.308876
2,0.119178,0.119214,-0.393070,0.507385,1.286118,0.176158,0.565010,2.225783,0.182919,1.942569,...,0.537095,0.656553,0.282480,0.282393,-16.179926,16.184928,1.175202,0.372602,-21.348525,67.334116
3,0.135533,-0.132967,-0.526513,0.410870,1.214796,0.177867,0.688087,1.960818,0.313265,1.676181,...,0.447717,0.601178,-0.239861,0.244583,-14.013578,-13.743013,1.217689,0.322938,-18.502983,69.768460
4,0.189646,0.064763,-0.792275,0.357015,1.170034,0.184028,0.896377,1.819974,0.667859,1.530305,...,0.401654,0.570359,0.079331,0.234196,-13.418416,4.545333,1.240117,0.292765,-16.774220,71.053458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469195,0.771160,-1.061284,1.235599,-0.150699,0.198219,-2.707576,2.872478,2.004618,3.247715,7.392969,...,2.711767,-0.886685,-0.629699,0.442182,-25.335189,-36.079082,0.072966,-0.055453,3.177212,4.180657
469196,0.769913,-1.016963,1.185170,-0.395387,-0.110361,-2.566486,2.746900,2.043041,3.031608,6.755359,...,2.596763,-1.024078,-0.623740,0.458035,-26.243449,-35.737662,-0.042474,-0.152717,8.750045,-2.433565
469197,0.804762,-1.000674,1.122728,-0.437428,-0.310409,-2.314917,2.729220,2.037639,2.909507,5.646540,...,2.355883,-1.020918,-0.626922,0.491331,-28.151182,-35.919995,-0.131005,-0.185140,10.607717,-7.506009
469198,0.771076,-0.904187,1.095665,-0.436282,-0.408929,-1.977082,2.474912,1.932815,2.612593,4.266417,...,2.024647,-0.940764,-0.593661,0.497292,-28.492706,-34.014293,-0.199294,-0.212823,12.193862,-11.418717


In [134]:
pred_list = []

for x in tdata:
    pred = model(x.float())
    pred_list.append(pred.sum().item())
    #print(x)
print(len(pred_list), pred_list)

782 [0.0, 0.0029020262882113457, 1.0, 0.999699592590332, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 2.265946806292242e-16, 3.082980715878904e-14, 1.0, 1.0, 1.0, 1.0, 1.0, 2.2950499295911682e-20, 5.087266913729641e-35, 1.0, 0.0, 2.104644236666786e-11, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.403900675242767e-05, 4.6705818563007215e-28, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 2.468677173794731e-08, 8.820344744890223e-19, 0.0, 1.0, 8.934699086733433e-28, 1.0, 0.0, 0.050419796258211136, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.6489729308912155e-15, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.188570982629026e-22, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.7405889630317688, 6.004354624712543e-21, 1.0, 1.0, 6.278165202713719e-38, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0075196517508496e-16, 1.0, 0.0010587862925603986, 1.0,

In [137]:
pred_list = np.array(pred_list)
pred_list = torch.from_numpy(pred_list)
pred_list

tensor([0.0000e+00, 2.9020e-03, 1.0000e+00, 9.9970e-01, 0.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 2.2659e-16, 3.0830e-14, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 2.2950e-20, 5.0873e-35, 1.0000e+00, 0.0000e+00,
        2.1046e-11, 1.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 4.4039e-05, 4.6706e-28, 0.0000e+00, 1.0000e+00, 1.0000e+00,
        0.0000e+00, 1.0000e+00, 1.0000e+00, 2.4687e-08, 8.8203e-19, 0.0000e+00,
        1.0000e+00, 8.9347e-28, 1.0000e+00, 0.0000e+00, 5.0420e-02, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+

In [148]:
predictionn = pred_list >= torch.FloatTensor([0.5])
print(predictionn)

tensor([False, False,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True, False, False,  True,  True,  True,  True,  True,
         True,  True,  True, False,  True, False, False,  True,  True,  True,
         True, False, False,  True,  True,  True,  True,  True, False, False,
         True, False, False,  True, False,  True,  True,  True,  True, False,
         True,  True,  True,  True,  True, False, False, False,  True,  True,
        False,  True,  True, False, False, False,  True, False,  True, False,
        False,  True,  True,  True,  True,  True,  True, False, False,  True,
         True,  True,  True,  True,  True,  True, False, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True, False,  True,  True, False,  True,  True,  True,  True,
        False,  True,  True, False,  True,  True, False,  True,  True,  True,
         True,  True,  True,  True, False,  True, False,  True, 

In [150]:
prediction = pd.DataFrame(l)
prediction

,0
0,False
1,False
2,True
3,True
4,False
...,...
777,True
778,True
779,False
780,True


In [191]:
p = prediction
p.columns=['id']
p

pandas.core.frame.DataFrame

In [198]:
# 1(True)이 운동중이 아닌 것(26번), 0(False)이 운동중인 것

len(p[p['id'] == True]), len(p[p['id'] == False])

(596, 186)

# LSTM

In [ ]:
# 모델 생성
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
len_features = train.shape[1] # feature 갯수
X=tf.reshape(np.array(train.iloc[:,0:]),[-1, 600, len_features])
X.shape

In [ ]:
trainX = X
trainX.shape

In [ ]:
y = tf.keras.utils.to_categorical(y_train_total) 
y.shape

In [ ]:
#가벼운 모델 생성
model = Sequential()
model.add(LSTM(32, input_shape=(600,len_features)))
model.add(Dense(128, activation='relu'))
model.add(Dense(61, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X,y, epochs=30, batch_size=128, validation_split=0.2)

In [ ]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, len_features])
test_X.shape

In [ ]:
prediction=model.predict(test_X)

In [ ]:
prediction

In [ ]:
prediction.shape

In [ ]:
submission

In [ ]:
submission.iloc[:,1:]=prediction

In [ ]:
submission

In [ ]:
submission.to_csv('LSTM_base(jun).csv', index=False)